 # Dog Breed Identification (MobileNetV2 transfer learning)

 I wrote this notebook to follow the assignment requirements:
  - Keras / TensorFlow usage
  - Transfer learning with MobileNetV2 (ImageNet weights)
  - Fine-tuning capability
  - No hard-coded local absolute paths (data is streamed from a public GCS URL)
  - Stratified train/val/test split
  - Experiment logging (parameters + metrics)
  - History plots of accuracy & loss
  - At least 25 test samples with Data, True Label, Predicted Label (CSV + montage image)

 **Important**: this notebook streams the ZIP from the public URL you provided:
 https://storage.googleapis.com/mariptime_assignment3_data/dog-breed-identification.zip

In [1]:
# runtime / user parameters
GCS_PUBLIC_ZIP = "https://storage.googleapis.com/mariptime_assignment3_data/dog-breed-identification.zip"
# where to extract/run training (runtime-only, relative path)
RUNTIME_DIR = "runtime_data"   # not an absolute path; created in current working directory
OUTPUT_DIR = "results"         # artifacts (experiments.csv, plots, models, sample CSVs)
IMG_SIZE = 224
BATCH_SIZE = 32
SEED = 42
VAL_SIZE = 0.15
TEST_SIZE = 0.10
EPOCHS_HEAD = 6
EPOCHS_FINETUNE = 4
MIN_SAMPLE_PRED = 25

# small hyperparameter grid (expand if you want more tuning)
LR_LIST = [1e-3]               # head training LR
DROPOUT_LIST = [0.3]
UNFREEZE_LAST_N_LIST = [0, 50]  # 0 = no fine-tune; otherwise unfreeze last N layers of MobileNetV2

# create directories
import os
os.makedirs(RUNTIME_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("Runtime dir:", os.path.abspath(RUNTIME_DIR))
print("Output dir:", os.path.abspath(OUTPUT_DIR))


Runtime dir: d:\Classes\UTD\Senior Year\CS 4372.501 - Computational Methods for Data Scientists\Assignments\CS4372-Assignments\Assignment 3\runtime_data
Output dir: d:\Classes\UTD\Senior Year\CS 4372.501 - Computational Methods for Data Scientists\Assignments\CS4372-Assignments\Assignment 3\results


In [2]:
import io
import zipfile
import requests
import random
import math
import time
from pathlib import Path
from PIL import Image, ImageOps, ImageDraw, ImageFont

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, log_loss

# seeds for reproducibility
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.10.1


In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12511173606684529828
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5711593472
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15035366103519077984
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9"
xla_global_id: 416903419
]


In [4]:
# === GPU setup ===
print("Available GPUs:", tf.config.list_physical_devices('GPU'))

# If a GPU is detected, allow memory growth (prevents TensorFlow from grabbing all VRAM at once)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs detected.")
    except RuntimeError as e:
        print(e)
else:
    print("⚠️ No GPU detected — training will run on CPU.")


Available GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
1 Physical GPUs, 1 Logical GPUs detected.


In [5]:
zip_url = GCS_PUBLIC_ZIP
print("Downloading from:", zip_url)

r = requests.get(zip_url, stream=True)
r.raise_for_status()

zip_bytes = io.BytesIO()
# stream write to BytesIO
for chunk in r.iter_content(chunk_size=1024*1024):
    if chunk:
        zip_bytes.write(chunk)
zip_bytes.seek(0)
print("Downloaded", zip_bytes.getbuffer().nbytes, "bytes")

# extract
# with zipfile.ZipFile(zip_bytes) as zf:
#     def safe_extract(zipf, extract_to):
#         for member in zipf.namelist():
#             # keep extraction inside extract_to
#             member_path = os.path.normpath(os.path.join(extract_to, member))
#             if not member_path.startswith(os.path.abspath(extract_to)):
#                 raise Exception("Attempted path traversal in zip file")
#         zipf.extractall(extract_to)
#     safe_extract(zf, RUNTIME_DIR)
with zipfile.ZipFile(zip_bytes) as zf:
    zf.extractall(RUNTIME_DIR)
print("Extracted to", RUNTIME_DIR)
# show top-level contents
for p in sorted(Path(RUNTIME_DIR).glob("*"))[:20]:
    print(p)


Downloaded 724495926 bytes
Extracted to runtime_data
runtime_data\labels.csv
runtime_data\sample_submission.csv
runtime_data\test
runtime_data\train


In [ ]:
labels_csv_path = "labels.csv"
print("Using labels CSV:", labels_csv_path)

train_images_dir = "train"
print("Train images directory:", train_images_dir)


Using labels CSV: runtime_data\labels.csv
Train images directory: runtime_data\train


In [7]:
labels_df = pd.read_csv(labels_csv_path)
if 'id' not in labels_df.columns or 'breed' not in labels_df.columns:
    raise ValueError("labels CSV must contain 'id' and 'breed' columns")

labels_df['filepath'] = labels_df['id'].astype(str) + '.jpg'
labels_df['filepath'] = labels_df['filepath'].apply(lambda p: os.path.join(train_images_dir, p))

# filter missing
labels_df['exists'] = labels_df['filepath'].apply(os.path.exists)
missing = (~labels_df['exists']).sum()
if missing > 0:
    print(f"Warning: {missing} images referenced in CSV not found in images directory. They will be ignored.")
labels_df = labels_df[labels_df['exists']].reset_index(drop=True)
labels_df = labels_df[['filepath','breed']]

print("Total usable images:", len(labels_df))
labels_df.head()


Total usable images: 10222


,filepath,breed
0,runtime_data\train\000bec180eb18c7604dcecc8fe0...,boston_bull
1,runtime_data\train\001513dfcb2ffafc82cccf4d8bb...,dingo
2,runtime_data\train\001cdf01b096e06d78e9e5112d4...,pekinese
3,runtime_data\train\00214f311d5d2247d5dfe4fe24b...,bluetick
4,runtime_data\train\0021f9ceb3235effd7fcde7f753...,golden_retriever


## Stratified split into train / val / test


In [8]:
def stratified_split(df, val_size=0.15, test_size=0.10, seed=42):
    train_val_df, test_df = train_test_split(df, test_size=test_size, stratify=df['breed'], random_state=seed)
    adjusted_val = val_size / (1.0 - test_size)
    train_df, val_df = train_test_split(train_val_df, test_size=adjusted_val, stratify=train_val_df['breed'], random_state=seed)
    return train_df.reset_index(drop=True), val_df.reset_index(drop=True), test_df.reset_index(drop=True)

train_df, val_df, test_df = stratified_split(labels_df, val_size=VAL_SIZE, test_size=TEST_SIZE, seed=SEED)
print("Train:", len(train_df), "Val:", len(val_df), "Test:", len(test_df))
print("Unique breeds:", labels_df['breed'].nunique())


Train: 7665 Val: 1534 Test: 1023
Unique breeds: 120


## Data generators (augmentation for training)


In [9]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.15,
    height_shift_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
    brightness_range=(0.8, 1.2)
)
test_val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_gen = train_datagen.flow_from_dataframe(
    train_df, x_col='filepath', y_col='breed',
    target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE,
    class_mode='categorical', shuffle=True, seed=SEED
)
val_gen = test_val_datagen.flow_from_dataframe(
    val_df, x_col='filepath', y_col='breed',
    target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE,
    class_mode='categorical', shuffle=False
)
test_gen = test_val_datagen.flow_from_dataframe(
    test_df, x_col='filepath', y_col='breed',
    target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE,
    class_mode='categorical', shuffle=False
)

class_indices = train_gen.class_indices
idx_to_class = {v: k for k, v in class_indices.items()}
print("Number of classes:", len(class_indices))


Found 7665 validated image filenames belonging to 120 classes.
Found 1534 validated image filenames belonging to 120 classes.
Found 1023 validated image filenames belonging to 120 classes.
Number of classes: 120


## Utility: build MobileNetV2 transfer model


In [10]:
def build_model(num_classes, img_size=224, dropout_rate=0.3, base_trainable=False, unfreeze_last_n=0):
    input_shape = (img_size, img_size, 3)
    base = MobileNetV2(include_top=False, weights='imagenet', input_shape=input_shape)
    base.trainable = base_trainable
    inputs = layers.Input(shape=input_shape)
    x = base(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(256, activation='relu')(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs, outputs)

    if unfreeze_last_n and unfreeze_last_n > 0:
        base.trainable = True
        total = len(base.layers)
        freeze_up_to = max(0, total - unfreeze_last_n)
        for i, layer in enumerate(base.layers):
            layer.trainable = (i >= freeze_up_to)
    return model


In [11]:
# compute class weights mapped to indices used by the generator
breeds_sorted = sorted(class_indices.keys())
y_train = train_df['breed'].values
try:
    cw_vals = compute_class_weight(class_weight='balanced', classes=np.array(breeds_sorted), y=y_train)
    class_weight = {class_indices[breed]: float(w) for breed, w in zip(breeds_sorted, cw_vals)}
    print("Computed class weights.")
except Exception as e:
    print("Could not compute class weights:", e)
    class_weight = None


Computed class weights.


## Training loop: for each 
### hyperparameter combination, train head, optionally fine-tune, evaluate, and log


In [12]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

experiment_rows = []
run_id = 0

for lr in LR_LIST:
    for dropout in DROPOUT_LIST:
        for unfreeze_last_n in UNFREEZE_LAST_N_LIST:
            run_id += 1
            print(f"\n=== Run {run_id}: lr={lr}, dropout={dropout}, unfreeze_last_n={unfreeze_last_n} ===")
            tf.keras.backend.clear_session()
            model = build_model(num_classes=len(class_indices), img_size=IMG_SIZE, dropout_rate=dropout, base_trainable=False, unfreeze_last_n=0)
            model.compile(optimizer=optimizers.Adam(learning_rate=lr), loss='categorical_crossentropy', metrics=['accuracy'])
            model.summary()
            ckpt = os.path.join(OUTPUT_DIR, f"best_model_run_{run_id}.h5")
            cbs = [
                ModelCheckpoint(ckpt, save_best_only=True, monitor='val_loss', verbose=1),
                EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1),
                ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)
            ]
            t0 = time.time()
            hist_head = model.fit(train_gen, validation_data=val_gen, epochs=EPOCHS_HEAD, callbacks=cbs, class_weight=class_weight, verbose=2)
            t_head = time.time() - t0
            print(f"Head trained in {t_head:.1f}s")

            val_loss, val_acc = model.evaluate(val_gen, verbose=0)[0:2]
            print(f"After head: val_loss={val_loss:.4f}, val_acc={val_acc:.4f}")

            hist_finetune = None
            if unfreeze_last_n and unfreeze_last_n > 0:
                # set trainability on base layers
                base_layer = None
                # locate base MobileNetV2 inside model by finding a layer with many children (has attribute 'layers')
                for layer in model.layers:
                    if hasattr(layer, 'layers') and len(getattr(layer, 'layers')) > 0:
                        # check name pattern
                        if 'mobilenet' in layer.name.lower() or 'conv' in layer.name.lower() or 'block' in layer.name.lower():
                            base_layer = layer
                            break
                # fallback: assume model.layers[1] is base
                if base_layer is None and len(model.layers) > 1:
                    base_layer = model.layers[1]
                if base_layer is not None and hasattr(base_layer, 'layers'):
                    total = len(base_layer.layers)
                    freeze_up_to = max(0, total - unfreeze_last_n)
                    for i, bl in enumerate(base_layer.layers):
                        bl.trainable = (i >= freeze_up_to)
                else:
                    # fallback: set all layers trainable then re-freeze initial portion by name
                    for layer in model.layers:
                        layer.trainable = True
                # recompile with lower lr for fine-tuning
                model.compile(optimizer=optimizers.Adam(learning_rate=lr*0.1), loss='categorical_crossentropy', metrics=['accuracy'])
                print("Starting fine-tuning with last", unfreeze_last_n, "layers unfrozen.")
                t1 = time.time()
                hist_finetune = model.fit(train_gen, validation_data=val_gen, epochs=EPOCHS_FINETUNE, callbacks=cbs, class_weight=class_weight, verbose=2)
                t_finetune = time.time() - t1
                print(f"Fine-tune done in {t_finetune:.1f}s")

            # load best weights if checkpoint exists
            if os.path.exists(ckpt):
                model.load_weights(ckpt)

            # evaluate on test set
            print("Predicting on test set...")
            proba = model.predict(test_gen, verbose=1)
            preds = np.argmax(proba, axis=1)
            true_inds = test_gen.classes  # integer indices consistent with train_gen.class_indices
            test_acc = accuracy_score(true_inds, preds)
            try:
                test_logloss = log_loss(tf.keras.utils.to_categorical(true_inds, num_classes=len(class_indices)), proba)
            except Exception:
                test_logloss = float('nan')
            print(f"Test acc: {test_acc:.4f}, logloss: {test_logloss:.4f}")

            # save history plot (use finetune history if present, else head history)
            def plot_history(hist, out_path):
                plt.figure(figsize=(10,4))
                plt.subplot(1,2,1)
                plt.plot(hist.history.get('accuracy', []), label='train_acc')
                plt.plot(hist.history.get('val_accuracy', []), label='val_acc')
                plt.title('Accuracy')
                plt.legend()
                plt.subplot(1,2,2)
                plt.plot(hist.history.get('loss', []), label='train_loss')
                plt.plot(hist.history.get('val_loss', []), label='val_loss')
                plt.title('Loss')
                plt.legend()
                plt.tight_layout()
                plt.savefig(out_path)
                plt.close()

            hist_plot = os.path.join(OUTPUT_DIR, f"history_run_{run_id}.png")
            if hist_finetune:
                plot_history(hist_finetune, hist_plot)
            else:
                plot_history(hist_head, hist_plot)

            # sample at least MIN_SAMPLE_PRED rows from test set evenly
            n_sample = max(MIN_SAMPLE_PRED, min(len(test_df), MIN_SAMPLE_PRED))
            sample_idxs = np.linspace(0, len(test_df)-1, n_sample, dtype=int)
            sample_rows = []
            for idx in sample_idxs:
                fp = test_df.iloc[idx]['filepath']
                true_label = test_df.iloc[idx]['breed']
                pred_label = idx_to_class[preds[idx]]
                sample_rows.append({'filepath': fp, 'true_label': true_label, 'predicted_label': pred_label})
            sample_df = pd.DataFrame(sample_rows)
            sample_csv = os.path.join(OUTPUT_DIR, f"sample_predictions_run_{run_id}.csv")
            sample_df.to_csv(sample_csv, index=False)

            # create montage image of sample predictions
            def make_montage(df_rows, out_file, thumb=IMG_SIZE):
                cols = 5
                rows_n = math.ceil(len(df_rows)/cols)
                montage = Image.new('RGB', (cols*thumb, rows_n*thumb), color=(255,255,255))
                draw = ImageDraw.Draw(montage)
                for i, r in enumerate(df_rows):
                    try:
                        im = Image.open(r['filepath']).convert('RGB')
                        im = ImageOps.fit(im, (thumb, thumb), Image.LANCZOS)
                    except Exception:
                        im = Image.new('RGB', (thumb, thumb), (200,200,200))
                    x = (i % cols) * thumb
                    y = (i // cols) * thumb
                    montage.paste(im, (x,y))
                    # write text: True / Pred (use small rectangle for legibility)
                    text = f"T:{r['true_label']}\nP:{r['predicted_label']}"
                    draw.rectangle([x+3, y+3, x+thumb-3, y+30], fill=(0,0,0,120))
                    draw.text((x+6, y+4), text, fill=(255,255,255))
                montage.save(out_file)

            montage_path = os.path.join(OUTPUT_DIR, f"sample_predictions_run_{run_id}.png")
            make_montage(sample_rows, montage_path)

            # log experiment
            experiment_rows.append({
                'run_id': run_id,
                'lr': lr,
                'dropout': dropout,
                'unfreeze_last_n': unfreeze_last_n,
                'val_loss': float(val_loss),
                'val_acc': float(val_acc),
                'test_loss': float(test_logloss),
                'test_acc': float(test_acc),
                'ckpt_path': ckpt,
                'history_plot': hist_plot,
                'sample_csv': sample_csv,
                'sample_montage': montage_path
            })

            print(f"Run {run_id} complete: test_acc={test_acc:.4f}")
# write experiments CSV
experiments_df = pd.DataFrame(experiment_rows)
experiments_csv = os.path.join(OUTPUT_DIR, "experiments.csv")
experiments_df.to_csv(experiments_csv, index=False)
print("Experiments saved to", experiments_csv)



=== Run 1: lr=0.001, dropout=0.3, unfreeze_last_n=0 ===
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 256)               327936    
                                                                 
 den

ValueError: axes don't match array

## Show experiments table (top runs)

In [ ]:
experiments_df = pd.read_csv(os.path.join(OUTPUT_DIR, "experiments.csv"))
experiments_df.sort_values('test_acc', ascending=False, inplace=True)
experiments_df.reset_index(drop=True, inplace=True)
experiments_df.head(10)


## Display sample predictions from best run

In [ ]:
if len(experiments_df) == 0:
    print("No experiments found.")
else:
    best = experiments_df.iloc[0]
    print("Best run:", best.to_dict())
    sample_csv = best['sample_csv']
    sample_img = best['sample_montage']
    display_df = pd.read_csv(sample_csv)
    display(display_df.head(50))
    # show montage
    from IPython.display import Image, display as idisp
    idisp(Image(filename=sample_img, width=900))
